### import libraries/packages

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc

### read file

In [2]:
with open("data\\train.txt", 'r') as f:
    lines = f.read().split('\n')

# lines

### data processing

In [3]:
# put each line into list
train_set = []
for x in lines:
    train_set.append(x.split(' '))

# train_set

In [4]:
# find the largest id by flattening the list of list
flat_list = []
train_set_flattened = []

# function to flatten list of list
# sourced from https://geekflare.com/flatten-list-python/
def flatten_list(data):
    # iterating over the data
    for element in data:
        # checking for list
        if type(element) == list:
            # calling the same function with current element as new argument
            flatten_list(element)
        else:
            flat_list.append(element)

flatten_list(train_set)

for x in flat_list[:-1]: # remove the last empty string
    train_set_flattened.append(int(x))

max_id = max(train_set_flattened)
max_id

4092

In [5]:
# create a adjacency matrix to store the connections between authors
matrix = np.zeros( shape=( (max_id)+1, (max_id)+1 ) )
matrix.shape

(4093, 4093)

In [6]:
# fill in the matrix, 
# mark the matrix cell with 1 if there is a connection
for row in train_set:
    for index, x in enumerate(row):
        for y in row[index+1:]:
            matrix[int(x)][int(y)] = 1
            matrix[int(y)][int(x)] = 1

# np.sum(matrix)

In [7]:
# see the non-zero amount of connections between authors
rows, cols = np.where(matrix>0)
values = matrix[rows,cols]
positive_edges = list(zip(rows, cols, values.astype(int)))

In [8]:
max_edge = max(values)

In [9]:
rows, cols = np.where(matrix<1)
values = matrix[rows,cols]
negative_edges = list(zip(rows, cols, values.astype(int)))

In [10]:
len(positive_edges)

32070

In [11]:
len(positive_edges)

32070

In [12]:
len(negative_edges)

16720579

unbalanced positive and negative samples

try under-sampling


In [13]:
np.random.seed(seed=90051)
negative_edges_resampled = []
for i in range(len(positive_edges)):
    negative_edges_resampled.append( negative_edges[np.random.randint(0, len(negative_edges))] )

len(negative_edges_resampled)


32070

In [14]:
edges = positive_edges + negative_edges_resampled
len(edges)

64140

### train test split

In [15]:
X = []
y = []

for a,b,c in edges:
    X.append([a,b])
    y.append(c)

In [16]:
len(X)

64140

In [17]:
len(y)

64140

In [18]:
X = np.array(X)
y = np.array(y)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 90051)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))

Training set has 44898 instances. Test set has 19242 instances.


In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Build model

In [21]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(C=1e4,penalty='l2')
lr.fit(X_train,y_train)

LogisticRegression(C=10000.0)

In [22]:
from sklearn.metrics import roc_auc_score
pred = lr.predict_proba(X_test)
y_pred = [p[1] for p in pred]
acc=lr.score(X_test,y_test)
print("acc: ", acc)
auc=roc_auc_score(y_test,y_pred)
print("auc: ", auc)

acc:  0.49542667082423864
auc:  0.49912865282168084
